In [34]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../src')

import numpy as np
import pandas as pd
from pingouin import partial_corr
from pingouin import pairwise_corr

from corr_networks import alt_pairwise_correlations
from corr_networks import pairwise_polychoric_correlations
from corr_networks import precision_mat_to_partial_corr
from corr_networks import cov_mat_to_regularized_partial_corr
from corr_networks import my_pairwise_correlations
from corr_networks import pairwise_polychoric_correlations

from data_metadata import load_gss_sas 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
np.set_printoptions(precision=3, suppress=True)

In [66]:
arr = np.ones((20000))
nans = np.random.randint(0, 20000, size=(1000,))
arr[nans] = np.nan
arr = arr.reshape(5000, 4)

non_nan_mat = ~np.isnan(arr)
sample_count = np.logical_and(non_nan_mat[:, :, np.newaxis], non_nan_mat[:, np.newaxis, :]).sum(axis=0)
print(sample_count)

[[4757 4543 4521 4510]
 [4543 4771 4537 4523]
 [4521 4537 4754 4506]
 [4510 4523 4506 4743]]


In [55]:
# Define mean vector and covariance matrix
dim = 4
random_mat = 2 * np.random.rand(dim, dim) - 1 # get a matrix
random_cov_mat = np.dot(random_mat, random_mat.T) # make it pos semi-definite

std_deviations = np.sqrt(np.diag(random_cov_mat))
random_cor_mat = random_cov_mat / np.outer(std_deviations, std_deviations)
    
mean = np.zeros((dim,))  # Mean vector

# Generate random samples from the multivariate normal distribution
num_samples = 5000
samples = np.random.multivariate_normal(mean, random_cov_mat, size=num_samples)

sample_df = pd.DataFrame(samples)

# Print the first few samples

precision_matrix = np.linalg.inv(random_cov_mat)
partial_corr_mat = precision_mat_to_partial_corr(precision_matrix)

sample_df_ord = pd.DataFrame()

for var in list(range(dim)):
    num_ordinal_values = np.random.randint(2, 11)
    # signed = np.random.rand() > 0.5
    var_std = np.sqrt(random_cov_mat[var, var])

    interval_spread = np.random.rand() * var_std
    leftmost_border = interval_spread * ((num_ordinal_values - 2)/ 2) + (np.random.rand() - 0.5)
    
    cutoffs = interval_spread * np.arange(num_ordinal_values - 1) - leftmost_border
    cutoffs = np.concatenate(([-np.inf], cutoffs, [np.inf]))
    sample_df_ord[var] = pd.cut(sample_df[var], bins=cutoffs, labels=np.arange(num_ordinal_values)).cat.codes


print(sample_df.head())
print(sample_df_ord.head())

print("Cov mat")
print(random_cov_mat)

print("Corr mat")
print(random_cor_mat)

print("Partial corr mat")
print(partial_corr_mat)

          0         1         2         3
0 -0.819702 -0.745665  0.307007 -0.069048
1  0.293409  0.161794 -0.033141  0.065351
2 -0.947439 -0.487434  0.138029  0.056342
3 -1.106842 -1.504332 -0.938740  0.317523
4 -1.056787 -1.212312 -1.223326  0.804173
   0  1  2  3
0  0  0  2  0
1  2  4  2  1
2  0  0  2  1
3  0  0  1  2
4  0  0  1  3
Cov mat
[[ 1.814  0.595 -0.988  0.263]
 [ 0.595  0.663  0.303 -0.13 ]
 [-0.988  0.303  1.505 -0.523]
 [ 0.263 -0.13  -0.523  0.221]]
Corr mat
[[ 1.     0.542 -0.598  0.416]
 [ 0.542  1.     0.303 -0.341]
 [-0.598  0.303  1.    -0.906]
 [ 0.416 -0.341 -0.906  1.   ]]
Partial corr mat
[[ 1.     0.967 -0.937 -0.676]
 [ 0.967  1.     0.905  0.623]
 [-0.937  0.905  1.    -0.863]
 [-0.676  0.623 -0.863  1.   ]]


In [63]:
# polychoric procedure 
my_pairwise_correlations(list(range(dim)), sample_df, method="pearson")

[[5000 5000 5000 5000]
 [5000 5000 5000 5000]
 [5000 5000 5000 5000]
 [5000 5000 5000 5000]]
[[ 1.     0.516 -0.616  0.444]
 [ 0.516  1.     0.313 -0.342]
 [-0.616  0.313  1.    -0.908]
 [ 0.444 -0.342 -0.908  1.   ]]


([0, 1, 2, 3],
 array([[ 1.   ,  0.966, -0.937, -0.663],
        [ 0.966,  1.   ,  0.906,  0.612],
        [-0.937,  0.906,  1.   , -0.856],
        [-0.663,  0.612, -0.856,  1.   ]]))

In [26]:
polychoric_corr_mat = pairwise_polychoric_correlations([0, 1, 2, 3], sample_df_ord)
polychoric_partial_corr_mat = cov_mat_to_regularized_partial_corr(polychoric_corr_mat, alpha=0.1)
print("polychoric_partial_corr")
print(polychoric_partial_corr_mat)

div [[1.936 1.989 1.708 1.938]
 [1.989 2.043 1.755 1.991]
 [1.708 1.755 1.507 1.71 ]
 [1.938 1.991 1.71  1.941]]
[[-1.    -0.624 -0.     0.543]
 [-0.624 -1.    -0.256  0.425]
 [-0.    -0.256 -1.    -0.363]
 [ 0.543  0.425 -0.363 -1.   ]]
polychoric_partial_corr
[[ 1.    -0.624 -0.     0.543]
 [-0.624  1.    -0.256  0.425]
 [-0.    -0.256  1.    -0.363]
 [ 0.543  0.425 -0.363  1.   ]]


In [5]:
# pearson procedure 1 on original data
my_pairwise_correlations(list(range(dim)), sample_df, method="pearson")

([0, 1, 2, 3],
 array([[ 1.   , -0.922, -0.034,  0.894],
        [-0.922,  1.   , -0.177,  0.852],
        [-0.034, -0.177,  1.   , -0.191],
        [ 0.894,  0.852, -0.191,  1.   ]]))

In [9]:
# pearson procedure 2 on original data
pearson_corr_df, pearson_corr_mat = alt_pairwise_correlations(list(range(dim)), sample_df, "pearson")
pearson_corr_mat

array([[ 1.   , -0.922, -0.034,  0.894],
       [-0.922,  1.   , -0.177,  0.852],
       [-0.034, -0.177,  1.   , -0.191],
       [ 0.894,  0.852, -0.191,  1.   ]])

In [10]:
# spearman procedure 1 on original data
my_pairwise_correlations(list(range(dim)), sample_df, method="spearman")

([0, 1, 2, 3],
 array([[ 1.   , -0.864, -0.035,  0.82 ],
        [-0.864,  1.   , -0.216,  0.754],
        [-0.035, -0.216,  1.   , -0.245],
        [ 0.82 ,  0.754, -0.245,  1.   ]]))

In [12]:
# spearman procedure 2 on original data
spearman_corr_df, spearman_corr_mat = alt_pairwise_correlations(list(range(dim)), sample_df, "spearman")
spearman_corr_mat

array([[ 1.   , -0.864, -0.035,  0.82 ],
       [-0.864,  1.   , -0.216,  0.754],
       [-0.035, -0.216,  1.   , -0.245],
       [ 0.82 ,  0.754, -0.245,  1.   ]])

In [91]:
# spearman procedure 1 on ordinalized data
my_pairwise_correlations(list(range(dim)), sample_df_ord, method="spearman")

array([[ 1.        , -0.6166378 , -0.10222241, -0.36117204],
       [-0.6166378 ,  1.        , -0.00269639, -0.4182649 ],
       [-0.10222241, -0.00269639,  1.        ,  0.4415108 ],
       [-0.36117204, -0.4182649 ,  0.4415108 ,  1.        ]])

In [92]:
# spearman procedure 2 on ordinalized data
spearman_corr_df, spearman_corr_mat = pairwise_correlations(list(range(dim)), sample_df_ord, "spearman")
spearman_corr_mat

array([[ 1.        , -0.6166378 , -0.10222241, -0.36117204],
       [-0.6166378 ,  1.        , -0.00269639, -0.4182649 ],
       [-0.10222241, -0.00269639,  1.        ,  0.4415108 ],
       [-0.36117204, -0.4182649 ,  0.4415108 ,  1.        ]])

In [67]:
gss_file = "C:/Users/vicvi/big-datasets/social_values/GSS_sas/gss7222_r3.sas7bdat"
variable_list = ["YEAR", "VOTE68", "VOTE72", "PARTYID", "POLVIEWS", "HOMOSEX", "NATENRGY", "AFFRMACT", "CONJUDGE", "HELPPOOR", "NATSPAC", "NATENVIR", "NATCITY", "NATDRUG", "LETDIE1"]

df, meta = load_gss_sas(gss_file, variable_list)

In [45]:
(df["YEAR"] == 2020) | (df["YEAR"] == 2021)

0        False
1        False
2        False
3        False
4        False
         ...  
72385    False
72386    False
72387    False
72388    False
72389    False
Name: YEAR, Length: 72390, dtype: bool

In [54]:
df.loc[(df["YEAR"] == 2022), "LETDIE1"].value_counts(dropna=False)

LETDIE1
NaN    2371
1.0     882
2.0     291
Name: count, dtype: int64

In [32]:
vars, corr_mat = my_pairwise_correlations(["PARTYID", "POLVIEWS", "HOMOSEX", "NATENRGY", "AFFRMACT", "CONJUDGE"], df, method="polychoric")
print(["VOTE68", "VOTE72",  "PARTYID", "POLVIEWS", "HOMOSEX", "NATENRGY", "AFFRMACT", "CONJUDGE"])
print(vars)
print(corr_mat)

[[ 1.     0.393 -0.092  0.273  0.303 -0.025]
 [ 0.393  1.    -0.35   0.346  0.279 -0.004]
 [-0.092 -0.35   1.    -0.268 -0.112 -0.036]
 [ 0.273  0.346 -0.268  1.     0.134 -0.043]
 [ 0.303  0.279 -0.112  0.134  1.    -0.055]
 [-0.025 -0.004 -0.036 -0.043 -0.055  1.   ]]
in here
outer_product [[1.643 1.837 1.518 1.546 1.465 1.291]
 [1.837 2.054 1.697 1.729 1.638 1.443]
 [1.518 1.697 1.402 1.428 1.353 1.193]
 [1.546 1.729 1.428 1.455 1.379 1.215]
 [1.465 1.638 1.353 1.379 1.306 1.151]
 [1.291 1.443 1.193 1.215 1.151 1.015]]
div [[1.282 1.355 1.232 1.243 1.21  1.136]
 [1.355 1.433 1.303 1.315 1.28  1.201]
 [1.232 1.303 1.184 1.195 1.163 1.092]
 [1.243 1.315 1.195 1.206 1.174 1.102]
 [1.21  1.28  1.163 1.174 1.143 1.073]
 [1.136 1.201 1.092 1.102 1.073 1.007]]
[[-1.     0.293  0.085  0.166  0.216 -0.003]
 [ 0.293 -1.    -0.285  0.198  0.159  0.011]
 [ 0.085 -0.285 -1.    -0.18  -0.03  -0.048]
 [ 0.166  0.198 -0.18  -1.    -0.001 -0.048]
 [ 0.216  0.159 -0.03  -0.001 -1.    -0.053]
 [-0.003

In [73]:
corr_df, corr_mat = my_pairwise_correlations(["NATSPAC", "NATENVIR", "NATCITY", "NATDRUG", "LETDIE1"], df, method="spearman", partial=True, sample_threshold=0.4)
print(corr_mat)

[[0.53  0.514 0.478 0.509 0.29 ]
 [0.514 0.541 0.488 0.52  0.297]
 [0.478 0.488 0.501 0.485 0.276]
 [0.509 0.52  0.485 0.537 0.295]
 [0.29  0.297 0.276 0.295 0.491]]
[[38350 37220 34625 36847 20967]
 [37220 39147 35323 37631 21478]
 [34625 35323 36245 35091 19997]
 [36847 37631 35091 38880 21358]
 [20967 21478 19997 21358 35537]]
[[ 1.     0.056 -0.051 -0.062]
 [ 0.056  1.     0.229  0.137]
 [-0.051  0.229  1.     0.218]
 [-0.062  0.137  0.218  1.   ]]
[[ 1.     0.075 -0.053 -0.059]
 [ 0.075  1.     0.209  0.096]
 [-0.053  0.209  1.     0.19 ]
 [-0.059  0.096  0.19   1.   ]]


In [132]:
arr = np.arange(16).reshape(4, 4).astype(float)
arr[2, 3] = np.nan
arr[1, 1] = np.nan

In [24]:
pairwise_polychoric_correlations(["NATSPAC", "NATENVIR", "NATCITY"], df)

      NATSPAC  NATENVIR
1613      3.0       3.0
1614      3.0       2.0
1615      3.0       1.0
1616      3.0       1.0
1617      3.0       2.0
1618      2.0       3.0
1619      3.0       1.0
1620      3.0       1.0
1621      2.0       2.0
1622      1.0       1.0
1623      3.0       2.0
      NATSPAC  NATENVIR
1613      3.0       3.0
1614      3.0       2.0
1615      3.0       1.0
1616      3.0       1.0
1617      3.0       2.0
1618      2.0       3.0
1619      3.0       1.0
1620      3.0       1.0
1621      2.0       2.0
1622      1.0       1.0
1623      3.0       2.0
      NATSPAC  NATCITY
1613      3.0      3.0
1614      3.0      1.0
1615      3.0      2.0
1616      3.0      2.0
1617      3.0      2.0
1618      2.0      2.0
1619      3.0      1.0
1620      3.0      1.0
1621      2.0      NaN
1622      1.0      1.0
1623      3.0      1.0
      NATSPAC  NATCITY
1613      3.0      3.0
1614      3.0      1.0
1615      3.0      2.0
1616      3.0      2.0
1617      3.0      2.0
1618      

array([[ 1.        ,  0.07242169, -0.06146199],
       [ 0.07242169,  1.        ,  0.33136766],
       [-0.06146199,  0.33136766,  1.        ]])

In [21]:
pairwise_polychoric_correlations([0, 1, 2, 3], sample_df_ord)

      0  1
1000  0  6
1001  0  4
1002  1  2
1003  1  5
1004  0  6
1005  1  3
1006  0  4
1007  0  4
1008  0  6
1009  1  7
1010  1  5
      0  2
1000  0  9
1001  0  9
1002  1  7
1003  1  6
1004  0  5
1005  1  6
1006  0  6
1007  0  5
1008  0  8
1009  1  4
1010  1  6
      0  3
1000  0  0
1001  0  0
1002  1  3
1003  1  1
1004  0  0
1005  1  1
1006  0  1
1007  0  2
1008  0  0
1009  1  0
1010  1  0
      1  2
1000  6  9
1001  4  9
1002  2  7
1003  5  6
1004  6  5
1005  3  6
1006  4  6
1007  4  5
1008  6  8
1009  7  4
1010  5  6
      1  3
1000  6  0
1001  4  0
1002  2  3
1003  5  1
1004  6  0
1005  3  1
1006  4  1
1007  4  2
1008  6  0
1009  7  0
1010  5  0
      2  3
1000  9  0
1001  9  0
1002  7  3
1003  6  1
1004  5  0
1005  6  1
1006  6  1
1007  5  2
1008  8  0
1009  4  0
1010  6  0


array([[ 1.        , -0.45833325,  0.18715718,  0.26338053],
       [-0.45833325,  1.        , -0.1817286 , -0.48963291],
       [ 0.18715718, -0.1817286 ,  1.        , -0.43169041],
       [ 0.26338053, -0.48963291, -0.43169041,  1.        ]])

In [147]:
rows_to_keep = np.where(~np.bitwise_or.reduce(np.isnan(arr), axis=1))[0].reshape(-1, 1)

In [146]:
cols_to_keep = np.where(~np.bitwise_or.reduce(np.isnan(arr), axis=1))[0].reshape(1, -1)

In [148]:
arr[rows_to_keep,cols_to_keep]

array([[ 0.,  3.],
       [12., 15.]])